# [微调一个预训练模型](https://huggingface.co/learn/nlp-course/zh-CN/chapter3)


In [ ]:
import os
os.environ['HF_ENDPOINT'] = "https://hf-mirror.com"

import torch
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
sequences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "This course is amazing!",
]
batch = tokenizer(sequences, padding=True, truncation=True, return_tensors='pt')
batch["labels"] = torch.tensor([1, 1])
optimizer = AdamW(model.parameters())
loss = model(**batch).loss
loss.backward()
optimizer.step()

## 预处理数据
### 从模型中心（Hub）加载数据集

In [ ]:
from datasets import load_dataset

# 使用 MRPC 数据集中的 GLUE 基准测试数据集 作为我们训练所使用的数据集
raw_datasets = load_dataset("glue", "mrpc")
raw_datasets

In [ ]:
# 访问该数据集中的 raw_train_dataset 对象
raw_train_dataset = raw_datasets["train"]
raw_train_dataset[0]

In [ ]:
# 显示数据集每列的类型
raw_train_dataset.features
# Label（标签） 是一种 ClassLabel（分类标签） ，也就是使用整数建立起类别标签的映射关系。 
# 0 对应于 not_equivalent（非同义） ， 1 对应于 equivalent（同义） 

### 预处理数据集
需要将文本转换为模型能够理解的数字，通过 Tokenizer 完成。

In [ ]:
from transformers import AutoTokenizer

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# tokenizer_sentences_1 = tokenizer(raw_datasets["train"]["sentence1"])
# tokenizer_sentences_2 = tokenizer(raw_datasets["train"]["sentence2"])
inputs = tokenizer("This is the first sentence.", "This is the second one.")
# input_ids: 一个整数列表或二维列表（对于批量输入），表示文本被分词后的 token ID。每个 token ID 对应词汇表中的一个词或子词。
# attention_mask:一个与 input_ids 形状相同的列表，表示每个位置是否是有效数据（1 表示有效，0 表示填充）。
# token_type_ids: token类型ID(token_type_ids) 的作用就是告诉模型输入的哪一部分是第一句，哪一部分是第二句。
# 注意: 如果选择其他的 checkpoint，不一定具有 token_type_ids
inputs

In [ ]:
# 将 input_ids 中的 id 转换回文字
tokenizer.convert_ids_to_tokens(inputs["input_ids"])

In [ ]:
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

# 可以同时处理数据集的多个元素，而不是分别处理每个元素
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets

### 动态填充
负责在批处理中将数据整理为一个 batch 的函数称为 `collate` 函数 。这是一个可以在构建 `DataLoader` 时传递的一个参数，默认是一个将你的数据集转换为 PyTorch 张量并将它们拼接起来的函数（如果你的元素是列表、元组或字典，则会使用递归进行拼接）。


In [ ]:
# 查看一个 batch 中每个条目的长度，看起来长度是参差不齐的
samples = tokenized_datasets["train"][:8]
samples = {k: v for k, v in samples.items() if k not in ["idx", "sentence1", "sentence2"]}
[len(x) for x in samples["input_ids"]]

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
# 动态填充意味着这个 batch 都应该填充到长度为 67
batch = data_collator(samples)
{k: v.shape for k, v in batch.items()}


下面是数据预处理涉及到的所有代码

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## 使用Trainer API 微调模型
Transformers 提供了一个 `Trainer` 类，可以帮助你在数据集上微调任何预训练模型。
### Training

In [ ]:
from transformers import TrainingArguments
from transformers import AutoModelForSequenceClassification
from transformers import Trainer

# 在我们定义 Trainer 之前第一步要定义一个 TrainingArguments 类，它包含 Trainer 在
# 训练和评估中使用的所有超参数。
training_args = TrainingArguments("test-trainer")
# 定义模型参数
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)
# 开始微调
# 每 500 步报告一次训练损失
trainer.train()

### 评估
前面 `Trainer` 在训练过程中没有增加评估指标，训练的时候只会输出loss，无法直观的了解到模型的性能

In [ ]:
# 构建一个有用的 compute_metrics() 函数，必须接收一个 EvalPrediction 对象（它是
# 一个带有 predictions 和 label_ids 字段的参数元组），并将返回一个字符串映射到浮点数
# 的字典（字符串是返回的指标名称，而浮点数是其值）。
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

In [ ]:
import numpy as np
# predictions.predictions 的输出是数据集中每个元素的 logits，需要转换成可以和标签
# 进行比较的预测值
preds = np.argmax(predictions.predictions, axis=-1)

In [ ]:
import evaluate
# 现在可以将这些 preds 与标签进行比较
# 使用 Evaluate 库中的指标
# 我们可以像加载数据集一样轻松地加载与 MRPC 数据集关联的指标
metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

In [ ]:
# 把所有东西打包在一起，我们就得到了 compute_metrics() 函数
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

为了查看模型在每个训练周期结束时的好坏，下面是我们如何使用 `compute_metrics()` 函数定义一个新的 `Trainer`

In [ ]:
# 设置了一个新的 TrainingArguments ，其 evaluation_strategy 设置
# 为 epoch 并且创建了一个新模型
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
# 这一次，它将在每个 epoch 结束时在训练损失的基础上报告验证损失和指标。
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)
trainer.train()

## 一个完整的训练过程
上面是基于 `Trainer` 类实现的，下面我们将在不使用 `Trainer` 类的情况下实现与上一节相同的结果

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
from torch.utils.data import DataLoader
from transformers import AutoModelForSequenceClassification
from transformers import AdamW
from transformers import get_scheduler
from tqdm.auto import tqdm
import evaluate

# step 1: 数据预处理
raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
# 设置数据集的格式，使其返回 PyTorch 张量而不是列表
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names


train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
)

""" 测试代码 """
"""
# 为了快速检验数据处理中没有错误，我们可以这样检验其中的一个 batch
for batch in train_dataloader:
    break
print({k: v.shape for k, v in batch.items()})
"""


# step 2: 创建模型
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

# step 3: 优化器
optimizer = AdamW(model.parameters(), lr=5e-5)

# step 4: 学习率调度器
# 从最大值 （5e-5） 到 0 的线性衰减
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

# step 5: 训练循环
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
print(device)
progress_bar = tqdm(range(num_training_steps))
model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

# step 6: 评估循环
metric = evaluate.load("glue", "mrpc")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()